In [31]:
import yaml
import json
import os
import math

path = os.getcwd() + '/topologies/second_subgraph/'
configs = os.listdir(path)

with open('data/resultData_fixed_ids.json','r') as f:
    data = json.load(f)

nodes = {}
for node in data['nodeList']:
    nodes[node['id']] = node

sources = [] # access
destinations = [] # kernel or mixed 

for config in configs:
    id = int(config.split('.')[0])

    if nodes[id]['deviceLevel'] == 'Access':
        sources.append(id)
    else:
        destinations.append(id) 

print("src:",len(sources), "\ndst:",len(destinations))

src: 1670 
dst: 33


In [32]:
SIMULATION_TIME = 100                   # (seconds)
PAIRS_PER_SRC = {'mu': 3, 'sigma': 1}   # (seconds)
MSG_FREQUENCY = 2                       # exponential dist (seconds)
MSG_SIZE = 20_000                       # (bytes)
PACKET_SIZE = 1400                      # (bytes)
BANDWIDTH = 10_000_000                  # (bytes)
PRIO_LEVELS = 3                         # Possible message priorities for QoS

In [33]:
import random

connections = {}
for src in sources:
    num_pairs = int(random.normalvariate(**PAIRS_PER_SRC))
    while num_pairs <= 0:
        num_pairs = int(random.normalvariate(**PAIRS_PER_SRC))
        
    connections[src] = random.sample(sources, k=num_pairs)


In [34]:
'''
    Generation Logic:
        - For each source - while the current message time is less than the simulation time
            - pick the next message time (exp dist)
            - pick a random dest (uniform)
            - pick a random message (size)
            - split the message into packets based on packets size
            - add packets accumulating transmission time (terminal -> switch)
'''

packets = []
unique_packet_id = 0
message_id = 0
for src, pairs in connections.items():
    current_time = random.expovariate(1/MSG_FREQUENCY)
    while current_time <= SIMULATION_TIME:
        dst = random.choice(pairs)
        message_size = random.expovariate(1/MSG_SIZE)
        
        num_packets = math.ceil(message_size / PACKET_SIZE) # ceil -> padding last packet to always be PACKET_SIZE
        
        packet_time = current_time
        for _ in range(num_packets):
            packets.append(
                {   
                    'unique_id': unique_packet_id,
                    'message_id': message_id,
                    'src': src,
                    'dst': dst,
                    'size': PACKET_SIZE,
                    'timestamp': packet_time
                }
            )
            packet_time += PACKET_SIZE / BANDWIDTH
            unique_packet_id += 1
  
        current_time += random.expovariate(1/MSG_FREQUENCY)
        message_id += 1
    

In [35]:
with open("trace_second_SG.txt", 'w+') as f:
    for packet in packets:
        packet_info = f"{str(packet['unique_id'])} {packet['message_id']} {str(packet['src'])} {str(packet['dst'])} {str(packet['size'])} {str(packet['timestamp'])}\n"
        f.write(packet_info)
